### データセットの取得と活用

Index of /jawiki
https://dumps.wikimedia.org/jawiki/

データセットはXMLやSQL、プレーンテキストなどの形式で提供されている。<br/>
ファイルによっては bzip(*.bz2)、7z(*.7z)、gzip(*.gz)などの形式で圧縮されているため使用時に解凍して使用する。

In [ ]:
#!wget https://dumps.wikimedia.org/jawiki/20170901/jawiki-20170901-pages-articles.xml.bz2

In [ ]:
#!bzcat jawiki-20170901-pages-articles.xml.bz2 | head -n 10

In [ ]:
# Wikipediaのデータセットから文章を抽出する

# 準備
# Wikipediaは独自のマークアップ(MediaWiki)でかかれているのでマークアップを取り除く

# WikiExtractor

!wget https://github.com/attardi/wikiextractor/raw/master/WikiExtractor.py


In [1]:
!cat WikiExtractor.py


#!/usr/bin/env python
# -*- coding: utf-8 -*-

# =============================================================================
#  Version: 2.75 (March 4, 2017)
#  Author: Giuseppe Attardi (attardi@di.unipi.it), University of Pisa
#
#  Contributors:
#   Antonio Fuschetto (fuschett@aol.com)
#   Leonardo Souza (lsouza@amtera.com.br)
#   Juan Manuel Caicedo (juan@cavorite.com)
#   Humberto Pereira (begini@gmail.com)
#   Siegfried-A. Gevatter (siegfried@gevatter.com)
#   Pedro Assis (pedroh2306@gmail.com)
#   Wim Muskee (wimmuskee@gmail.com)
#   Radics Geza (radicsge@gmail.com)
#   orangain (orangain@gmail.com)
#   Seth Cleveland (scleveland@turnitin.com)
#   Bren Barn
#
# =============================================================================
#  Copyright (c) 2011-2017. Giuseppe Attardi (attardi@di.unipi.it).
# =============================================================================
#  This file is part of Tanl.
#
#  Tanl is free software; you can redistribute it and/or modify i

In [ ]:
# WikiExtractor
# Wikipediaのダンプファイルをテキストに変換出来る。
# オプション
# --no-templates ページの冒頭などに貼られるテンプレートを展開しない
# -o 出力先のディレクトリを指定
# -b 分割するファイルのサイズ(この例では100MB)を指定

# !python WikiExtractor.py --no-templates -o articleDump -b 100G jawiki-20170901-pages-articles.xml.bz2

In [ ]:
!brew install mecab mecab-ipadic

In [2]:
!mecab -v

mecab of 0.996



In [ ]:
!pip install mecab-python3

In [3]:
import MeCab

tagger = MeCab.Tagger()
tagger.parse('')  # これは .parseToNode() の不具合を回避するために必要。

# .parseToNode() で最初の形態素を表すNodeオブジェクトを取得する。
node = tagger.parseToNode('すもももももももものうち')

while node:
    # .surfaceは形態素の文字列、.featureは品詞などを含む文字列をそれぞれ表す。
    print(node.surface, node.feature)
    node = node.next  # .nextで次のNodeを取得する。

 BOS/EOS,*,*,*,*,*,*,*,*
すもも 名詞,一般,*,*,*,*,すもも,スモモ,スモモ
も 助詞,係助詞,*,*,*,*,も,モ,モ
もも 名詞,一般,*,*,*,*,もも,モモ,モモ
も 助詞,係助詞,*,*,*,*,も,モ,モ
もも 名詞,一般,*,*,*,*,もも,モモ,モモ
の 助詞,連体化,*,*,*,*,の,ノ,ノ
うち 名詞,非自立,副詞可能,*,*,*,うち,ウチ,ウチ
 BOS/EOS,*,*,*,*,*,*,*,*


In [ ]:
import sys
import os
from glob import glob
from collections import Counter

import MeCab


def main():
    """
    コマンドライン引数で指定したディレクトリ内のファイルを読み込んで、頻出単語を表示する。
    """

    input_dir = sys.argv[1]  # コマンドラインの第1引数で、WikiExtractorの出力先のディレクトリを指定する。

    tagger = MeCab.Tagger('')
    tagger.parse('')  # parseToNode() の不具合を回避するために必要。
    # 単語の頻度を格納するCounterオブジェクトを作成する。
    # Counterクラスはdictを継承しており、値としてキーの出現回数を保持する。
    frequency = Counter()
    count_proccessed = 0

    # glob()でワイルドカードにマッチするファイルのリストを取得し、マッチしたすべてのファイルを処理する。
    for path in glob(os.path.join(input_dir, '*', 'wiki_*')):
        print('Processing {0}...'.format(path), file=sys.stderr)

        with open(path) as file:  # ファイルを開く。
            for content in iter_docs(file):  # ファイル内の全記事について反復処理する。
                tokens = get_tokens(tagger, content)  # ページから名詞のリストを取得する。
                # Counterのupdate()メソッドにリストなどの反復可能オブジェクトを指定すると、
                # リストに含まれる値の出現回数を一度に増やせる。
                frequency.update(tokens)

                # 10,000件ごとに進捗を表示。
                count_proccessed += 1
                if count_proccessed % 10000 == 0:
                    print('{0} documents were processed.'.format(count_proccessed),
                          file=sys.stderr)

    # 全記事の処理が完了したら、上位30件の名詞と出現回数を表示する。
    for token, count in frequency.most_common(30):
        print(token, count)


def iter_docs(file):
    """
    ファイルオブジェクトを読み込んで、記事の中身（開始タグ <doc ...> と終了タグ </doc> の間のテキスト）を
    順に返すジェネレーター関数。
    """

    for line in file:
        if line.startswith('<doc '):
            buffer = []  # 開始タグが見つかったらバッファを初期化する。
        elif line.startswith('</doc>'):
            # 終了タグが見つかったらバッファの中身を結合してyieldする。
            content = ''.join(buffer)
            yield content
        else:
            buffer.append(line)  # 開始タグ・終了タグ以外の行はバッファに追加する。


def get_tokens(tagger, content):
    """
    文書内に出現した名詞のリストを取得する関数。
    """

    tokens = []  # この記事で出現した名詞を格納するリスト。

    node = tagger.parseToNode(content)
    while node:
        # node.featureはカンマで区切られた文字列なので、split()で分割して
        # 最初の2項目をcategoryとsub_categoryに代入する。
        category, sub_category = node.feature.split(',')[:2]
        # 固有名詞または一般名詞の場合のみtokensに追加する。
        if category == '名詞' and sub_category in ('固有名詞', '一般'):
            tokens.append(node.surface)
        node = node.next

    return tokens

if __name__ == '__main__':
    main()

In [ ]:
# !python ../sample_codes/5-1/word_frequency.py articles/

In [ ]:
# NEologd
# https://github.com/neologd/mecab-ipadic-neologd